In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import os

# Set path to dataset
base_dir = '/content/drive/MyDrive/Rice Leaf Diseases Dataset'  # Update with path


Mounted at /content/drive


In [2]:
# Set up ImageDataGenerator for augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Normalize pixel values between 0 and 1
    rotation_range=30,   # Rotate images
    width_shift_range=0.2, # Shift image width
    height_shift_range=0.2, # Shift image height
    shear_range=0.2,     # Shear transformations
    zoom_range=0.2,      # Zoom in/out
    horizontal_flip=True, # Random horizontal flips
    fill_mode='nearest',  # Fill mode for new pixels
    validation_split=0.2  # Split training into train/validation
)

# Set up testing generator
test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for test data

# Define batch size
batch_size = 32
img_size = (224, 224)  # EfficientNet-B3 default input size is 224x224

# Load the dataset from directories and augment
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # This is the training set
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # This is the validation set
)

# For testing, no augmentation
test_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Test data doesn't need to be shuffled
)


Found 6946 images belonging to 4 classes.
Found 1735 images belonging to 4 classes.
Found 8681 images belonging to 4 classes.


In [9]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [10]:
!pip install --upgrade tensorflow

In [11]:
# Clear previous session to avoid shape issues
from tensorflow.keras import backend as K
K.clear_session()

# Import necessary libraries
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input
from tensorflow.keras.models import Model

# Define input shape and create the EfficientNetB3 base model
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Load the EfficientNetB3 model without the top layers
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_tensor=inputs)

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of EfficientNetB3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(4, activation='softmax')(x)  # 4 classes: Brown Spot, Leaf Blast, Bacterial Leaf Blight, Healthy

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to check layers and shapes
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 40)   │          1,080 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 40)   │            160 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 40)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 40)   │            360 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 40)   │            160 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 40)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 40)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 40)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 10)       │            410 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 11,178,035 (42.64 MB)

 Trainable params: 394,500 (1.50 MB)

 Non-trainable params: 10,783,535 (41.14 MB)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20  # Train for 20 epochs
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


217/217 ━━━━━━━━━━━━━━━━━━━━ 7041s 32s/step - accuracy: 0.2571 - loss: 1.4232 - val_accuracy: 0.2784 - val_loss: 1.3838
Epoch 2/20
  1/217 ━━━━━━━━━━━━━━━━━━━━ 15:03 4s/step - accuracy: 0.2188 - loss: 1.4107

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


217/217 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.2188 - loss: 1.4107 - val_accuracy: 0.4286 - val_loss: 1.3501
Epoch 3/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 1511s 7s/step - accuracy: 0.2637 - loss: 1.4035 - val_accuracy: 0.2789 - val_loss: 1.3854
Epoch 4/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.1875 - loss: 1.3950 - val_accuracy: 0.2857 - val_loss: 1.3804
Epoch 5/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 1508s 7s/step - accuracy: 0.2569 - loss: 1.3958 - val_accuracy: 0.2784 - val_loss: 1.3844
Epoch 6/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2500 - loss: 1.4155 - val_accuracy: 0.4286 - val_loss: 1.3697
Epoch 7/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 1514s 7s/step - accuracy: 0.2643 - loss: 1.3886 - val_accuracy: 0.2784 - val_loss: 1.3847
Epoch 8/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.2812 - loss: 1.3747 - val_accuracy: 0.4286 - val_loss: 1.3759
Epoch 9/20
217/217 ━━━━━━━━━━━━━━━━━━━━ 1523s 7s/step - accuracy: 0.2765 - loss: 1.3855 - val_accuracy: 0.2

In [13]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')


272/272 ━━━━━━━━━━━━━━━━━━━━ 1416s 5s/step - accuracy: 0.5608 - loss: 1.3800
Test Accuracy: 0.33


In [ ]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Fine-tune for additional 10 epochs
)


Epoch 1/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 4774s 22s/step - accuracy: 0.3141 - loss: 1.5409 - val_accuracy: 0.3032 - val_loss: 1.3769
Epoch 2/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.6562 - loss: 0.9770 - val_accuracy: 0.2857 - val_loss: 1.3824
Epoch 3/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 4719s 21s/step - accuracy: 0.5081 - loss: 1.1433 - val_accuracy: 0.3663 - val_loss: 1.3344
Epoch 4/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 41s 96ms/step - accuracy: 0.6875 - loss: 0.9832 - val_accuracy: 0.2857 - val_loss: 1.6171
Epoch 5/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 4686s 21s/step - accuracy: 0.6258 - loss: 0.9132 - val_accuracy: 0.5723 - val_loss: 1.1439
Epoch 6/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.5938 - loss: 0.8397 - val_accuracy: 0.8571 - val_loss: 0.9971
Epoch 7/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 4583s 21s/step - accuracy: 0.7215 - loss: 0.7412 - val_accuracy: 0.6389 - val_loss: 1.0425
Epoch 8/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.7812 - loss: 0.5165 -